In [1]:
# NOT TESTED

# MUST BE RUN IN GENET KERNEL

from genet import read_gtfs
import genet as gn
import pandas as pd
from datetime import datetime, timedelta
import logging
import os
from genet.schedule_elements import Route, Service, Stop
from genet import read_gtfs

# Function to make insert new station, from https://github.com/arup-group/transport-east-network/blob/main/scenarios/SIP_future/pt_schemes.py
def insert_new_station(network, df, new_station_id, new_station_name, lon, lat, coord_sys, new_offset,
                       station_served_1, station_served_2, station_served_3,
                       closest_existing_station_id_fromLondon, first_stop_id):
    # Function to include a new station at existing routes without impacting the original schedule times at existing stops
    # df: dataframe built using GTFS feed in genet
    # new_station_id: unique ID of the new station
    # new_station_name: name of new station
    # lat: latitude of new station
    # lon: longitude of new station
    # coord_sys: coordinate system
    # new_offset: differential offset from previous station
    # station_served_1: station 1 that existing route serves
    # station_served_2: station 2 that existing route serves
    # station_served_3: station 3 that existing route serves
    # closest_existing_station_id_fromLondon: closest existing station - previous station to new station when train comes from London
    # first_stop_id: id of first stop when headway 1 (!= headway 2)

    # Get a list of stations served by each trip (trip_id)
    grouped_df = df.groupby('service_id')['to_stop_name'].unique().reset_index()

    # Filter the dataframe for routes passing through "Liverpool London St" and...
    filtered_df = grouped_df[grouped_df['to_stop_name'].apply(lambda x: station_served_1 in x)]
    filtered_df = filtered_df[filtered_df['to_stop_name'].apply(lambda x: station_served_2 in x)]
    filtered_df = filtered_df[filtered_df['to_stop_name'].apply(lambda x: station_served_3 in x)]

    # Get the unique route_ids
    unique_service_ids = filtered_df['service_id'].unique()

    # Convert offset to datetime
    new_offset = datetime.strptime(new_offset, '%H:%M:%S')

    new_stop = Stop(id=new_station_id, x=lon, y=lat, name=new_station_name, epsg=coord_sys)
    new_services = []
    route_ids = set()

    # Loop through services where new stop will be added
    for service_id_iteration in unique_service_ids:
        specific_routes = set(network.schedule[service_id_iteration].route_ids())
        # store the route IDs that will be removed
        route_ids |= specific_routes
        # store new Routes
        new_routes = []

        # Loop through routes where new stop will be added
        for route_id_iteration in specific_routes:
            logging.info(f"Processing route {route_id_iteration} of service {service_id_iteration}")
            # Select an existing route
            existing_route = network.schedule.route(route_id_iteration)
            route_short_name = existing_route.route_short_name
            mode = existing_route.mode
            trips = existing_route.trips
            original_arrival_offsets = existing_route.arrival_offsets
            original_departure_offsets = existing_route.departure_offsets
            id = existing_route.id
            original_stops = list(existing_route.stops())
            original_stops_name = existing_route.ordered_stops

            if closest_existing_station_id_fromLondon in original_stops_name:

                if original_stops_name[0] == first_stop_id:
                    closest_station_position = original_stops_name.index(closest_existing_station_id_fromLondon)
                else:
                    closest_station_position = original_stops_name.index(closest_existing_station_id_fromLondon) - 1

                # Calculate offset at new station from previous station
                original_offset = datetime.strptime(original_departure_offsets[closest_station_position], '%H:%M:%S')
                new_stop_offset = (original_offset + timedelta(hours=new_offset.hour, minutes=new_offset.minute,
                                                               seconds=new_offset.second)).strftime('%H:%M:%S')

                # Define a new route using the characteristics of the existing route and the new stop
                arrival_offsets = original_arrival_offsets[:closest_station_position + 1] + [
                    new_stop_offset] + original_arrival_offsets[closest_station_position + 1:]
                departure_offsets = original_departure_offsets[:closest_station_position + 1] + [
                    new_stop_offset] + original_departure_offsets[closest_station_position + 1:]

                # Define new route attributes
                new_routes.append(Route(
                    route_short_name=route_short_name,
                    mode=mode,
                    trips=trips,
                    arrival_offsets=arrival_offsets,
                    departure_offsets=departure_offsets,
                    id=id + "_NEW",
                    stops=original_stops[:closest_station_position + 1] + [new_stop] + original_stops[
                                                                                       closest_station_position + 1:],
                ))
            else:
                # Define new route attributes if the route does not pass through the new station but is but of the list of services that pass through the new station to avoid its exclusion. This route will have exactly the same attributes as the original route.
                new_routes.append(Route(
                    route_short_name=route_short_name,
                    mode=mode,
                    trips=trips,
                    arrival_offsets=original_arrival_offsets,
                    departure_offsets=original_departure_offsets,
                    id=id + "_NEW",
                    stops=original_stops,
                ))            

        # add new service with the new routes
        new_services.append(
            Service(
                id=network.schedule[service_id_iteration].id + "_NEW",
                routes=new_routes,
                name=network.schedule[service_id_iteration].name
            )
        )

    logging.info("Applying changes to the network and schedule")
    network.schedule.remove_routes(route_ids)
    network.schedule.add_services(new_services)
                           
    network.teleport_service(service_ids=[s.id for s in new_services])
    #Include re-routing as below (and delete teleport_service line above) in case the services have been already included previously (only necessary for fixes of this scenario)
    #network.schedule.remove_routes(route_ids)
    #network.schedule.add_services(new_services)
                           
    logging.info("Reindexing Services and Routes to their original ID to help with analysis")
    for service in new_services:
        service.reindex(service.id.replace("_NEW", ""))
        for route in service.routes():
            route.reindex(route.id.replace("_NEW", ""))
    logging.info(f"Adding new station: `{new_station_id}` is complete.")
    return network


/home/siskiyoucedar/micromamba/envs/genet/lib/python3.12/site-packages/keplergl/keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [ ]:
from genet import read_gtfs
import genet as gn
import pandas as pd
from datetime import datetime, timedelta
import logging
import os
from genet.schedule_elements import Route, Service, Stop
from genet import read_gtfs

# Read network to use for changes
path_to_matsim_network = "/mnt/c/_BERTIE_data/2_2040_do_minimum_40WFH/"
network = os.path.join(path_to_matsim_network, "output_network.xml")
schedule = os.path.join(path_to_matsim_network, "output_transitSchedule.xml")
vehicles = os.path.join(path_to_matsim_network, "output_transitVehicles.xml")
n = gn.read_matsim(
    path_to_network=network, epsg="epsg:27700", path_to_schedule=schedule, path_to_vehicles=vehicles
)

In [ ]:
df_stop_times = n.schedule.trips_with_stops_to_dataframe(gtfs_day="20190603")

# 51.892978, 0.830467 - BEACON END, COLCHESTER; LONDON TO NORWICH
insert_new_station(
    n, df_stop_times, 'BeaconEndNEW', 'Beacon End Station',
    594860, 225389, 'epsg:27700', '00:02:00',
    'Marks Tey', 'Colchester', 'London Liverpool Street',
    '910GMRKSTEY', '910GLIVST')


logging.info("Adding car and bike access for BeaconEndNEW")
n.schedule.apply_attributes_to_stops(
    {
        "BeaconEndNEW": {
            "attributes": {
                "bikeAccessible": "true",
                "carAccessible": "true",
                "accessLinkId_car": "65610" 
            }
        }
    }
)

# 51.901987, 0.936787 - PARSONS HEATH, COLCHESTER; LONDON TO NORWICH
insert_new_station(
    n, df_stop_times, 'ParsonsHeathNEW', 'Parsons Heath Station',
    602134, 226681, 'epsg:27700', '00:02:00',
    'Colchester', 'Manningtree', 'London Liverpool Street',
    '910GCLCHSTR', '910GLIVST')

logging.info("Adding car and bike access for ParsonsHeathNEW")
n.schedule.apply_attributes_to_stops(
    {
        "ParsonsHeathNEW": {
            "attributes": {
                "bikeAccessible": "true",
                "carAccessible": "true",
                "accessLinkId_car": "5177175557524539835_5177175557540289769"
            }
        }
    }
)

# 52.035361, 1.207435 - WARREN HEATH, IPSWICH; IPSWICH TO FELIXSTOWE
insert_new_station(
    n, df_stop_times, 'WarrenHeathNEW', 'Warren Heath Station',
    620094, 242296, 'epsg:27700', '00:02:00',
    'Derby Road', 'Ipswich', 'Felixstowe',
    '910GDERBYRD', '910GIPSWICH')

logging.info("Adding car and bike access for WarrenHeathNEW")
n.schedule.apply_attributes_to_stops(
    {
        "WarrenHeathNEW": {
            "attributes": {
                "bikeAccessible": "true",
                "carAccessible": "true",
                "accessLinkId_car": "5177344938942946157_5177344939029126855"
            }
        }
    }
)

# 52.071381, 1.133138 - NORWICH ROAD, IPSWICH; IPSWICH TO FELIXSTOWE (but not Lowestoft; the logic being this provides more of a useful commuter service along w Warren Heath)
insert_new_station(
    n, df_stop_times, 'NorwichRoadNEW', 'Norwich Road Station',
    614826, 246078, 'epsg:27700', '00:03:00',
    'Westerfield', 'Derby Road', 'Felixstowe',
    '910GWSTRFLD', '910GFLXSTOW')

logging.info("Adding car and bike access for NorwichRoadNEW")
n.schedule.apply_attributes_to_stops(
    {
        "NorwichRoadNEW": {
            "attributes": {
                "bikeAccessible": "true",
                "carAccessible": "true",
                "accessLinkId_car": "246644"
            }
        }
    }
)

# 52.628904, 1.360724 - BROADLANDS, NORWICH; NORWICH TO GT YARMOUTH
insert_new_station(
    n, df_stop_times, 'BroadlandsNEW', 'Broadlands Station',
    627538, 308764, 'epsg:27700', '00:04:00',
    'Norwich', 'Brundall Gardens', 'Lowestoft',
    '910GBRUNDLG', '910GNRCH')

logging.info("Adding car and bike access for BroadlandsNEW")
n.schedule.apply_attributes_to_stops(
    {
        "BroadlandsNEW": {
            "attributes": {
                "bikeAccessible": "true",
                "carAccessible": "true",
                "accessLinkId_car": "570221"
            }
        }
    }
)

In [ ]:
n.schedule.validate_vehicle_definitions()

In [ ]:
n.schedule.change_log().tail()

In [ ]:
# Save the new network with added station - can skip to this code block if you only want to add some, not all stations
output_path = "/mnt/c/_BERTIE_data/2040_new_stations_test"

n.write_to_matsim(output_path)